In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacUser"
password = "cs340"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GS_Logo.png' # logo of the company
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.A([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),  #inserting the logo 
               height = 250, width = 251)], href = 'https://www.snhu.edu', target = "_blank"),   
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Chris Abbott CS 340 Project Two!'))), #identifying myself in the dashboard
    html.Hr(),
    html.Img(
        src = 'data:image/png;base64,{}'.format(encoded_image.decode()),   #inserting the logo
        style = {'height': '15%', 'width': '15%'}
    ),
    
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id = 'filter-type', 
            options = [
                {'label': 'Reset', 'value': 'RESET'},               #adding the buttons for filtering options
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here         
        editable = False,
        filter_action="native",
        sort_action = 'native',
        sort_mode = "multi",           #interactive data table with all of the features
        column_selectable = False,
        row_selectable = False,
        row_deletable = False, 
        selected_columns = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.readAll({}))
    elif filter_type == 'WATER':
        df = pd.DataFrame(list(shelter.readAll({     #filtering the animals by skill
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26},
            "age_upon_outcome_in_weeks": {"$lte": 156},  #setting the traits that fall into this
            "outcome_type": {"$nin": ["Euthanasia"]}     # type of skill set
            
        })))
    elif filter_type == "MOUNTAIN OR WILDERNESS":      #filtering the animals by skill
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottwieler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26},
            "age_upon_outcome_in_weeks": {"$lte": 156},    #setting the traits that fall into this
            "outcome_type": {"$nin": ["Euthanasia"]}       # type of skill set
            
        })))
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING': #filtering the animals by skill
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20},
            "age_upon_outcome_in_weeks": {"$lte": 300},    #setting the traits that fall into this
            "outcome_type": {"$nin": ["Euthanasia"]}       # type of skill set
            
        })))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(Output('datatable-id', 'style_data_conditional'),
            [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
Output('graph-id', "children"),
[Input('datatable-id', "derived_viewport_data")])
# # ###FIX ME ####
#     # add code for chart of your choice (e.g. pie chart) #
#     #return [
#     #    dcc.Graph(            
#     #        figure = ###
#     #    )    
#     #]
def update_graphs(viewData):
     dff = pd.DataFrame.from_dict(viewData)
     
     return [
        dcc.Graph(figure = px.pie(dff, names = 'breed',) #the value on the pie chart
                 )       
     ]

  

@app.callback(
Output('map-id', "children"),
[Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
        dff = pd.DataFrame.from_dict(viewData)
        return [
            dl.Map(style = {'width': '1000px','height': '500px'}, #size of the map displayed
            center = [30.75, -97.48],zoom = 10, children = [
                dl.TileLayer(id = "base-layer-id"),
                dl.Marker(position = [30.75, -97.48], children = [  #position of the marker on the map
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),  #what the pin on the map is called
                        html.P(dff.iloc[1,9])
                        
                    ])
                ])
                         ]) 
        ]               
app             

username and password are:  aacUser cs340
Connection was successful
